In [114]:
import sys
sys.path.append('../')
from pdf2df import convert_to_df
# df = convert_to_df('../../pdf_issue/dem_1.pdf')
df = convert_to_df("../../pdf_data/Demesne/Demesne.pdf")

df

{'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Rave Reports (http://www.nevrona.com/rave)', 'producer': 'Nevrona Designs', 'creationDate': 'D:20220412094050Z', 'modDate': "D:20220413121438+01'00'", 'trapped': '', 'encryption': None}


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,INVOICE,488,20,559,42,0,0,1
1,DEMESNE,19,21,78,38,0,1,1
2,ELECTRICAL,81,21,156,38,0,1,1
3,SALES,159,21,197,38,0,1,1
4,LTD,200,21,224,38,0,1,1
...,...,...,...,...,...,...,...,...
260,Account,48,802,81,814,60,0,1
261,:,83,802,86,814,60,0,1
262,83020698,89,802,130,814,60,0,1
263,Sort,132,802,149,814,60,0,1


In [115]:
df_sort= df.sort_values(by=['page_no','Y1','X1']).reset_index(drop=True)


In [116]:
def rearrange(df):
    for i in range(1, len(df)):
        temp=df.loc[i,'Y1'] - df.loc[i-1, 'Y1']
        if -5 < temp < 5:
            df.loc[i,'Y1'] = df.loc[i-1,'Y1']
    
    return df.sort_values(by=['page_no','Y1','X1']).reset_index(drop=True)
    

In [117]:
df = rearrange(df_sort)
df.to_csv('../../pdf_issue/dem_1.csv')

In [118]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [119]:
pos = find_next_word(df,'Invoice', 'Number')
df[pos[0]+2:pos[0]+3]['TEXT'].values[0]
# invoice_no = df[pos[0]+2:pos[0]+3]['TEXT'].values[0]

'1112873'

In [120]:
pos = find_next_word(df,'Invoice','Date')
df.iloc[pos[0]+2:pos[0]+3]['TEXT'].values[0]

'12/04/2022'

In [121]:
def get_table_data(df):
    
    result = list()
    page_no = df['page_no'].unique()
    try:
        for page in page_no:
            mydf = df[df['page_no'] == page].reset_index(drop=True)
            get_result(mydf, result)
    except Exception as e:
        traceback.print_exc()

    return result

In [122]:
def get_result(df, result):
    
    table_start = find_next_word(df, 'Unit','Price')
    table_end = find_next_word(df, 'Vat','Analysis')
    if not len(table_end):
        last_pos = df.iloc[[-1]].index.to_list()
        print(last_pos)
        table_data = df[table_start[0]:]

    else: 
        table_data = df[table_start[0]:table_end[0]] 
        
    msg = dict()

    for index, row in table_data.iterrows():
        if row['X1'] < 60 and row['TEXT'].isdigit():
            msg = dict()
            msg['qty'] = row['TEXT']
            msg['part'] = ''

        if ('qty' in msg) and (60 < row['X1'] < 90):
            msg['part'] = row['TEXT']
            msg['desc'] = ''

        if ('desc' in msg) and (180 < row['X1'] < 360):
            msg['desc'] = msg['desc'] + " " + row['TEXT']
            msg['unit'] = ''

        if ('unit' in msg) and (400 < row['X1'] < 460):
            msg['unit'] = row['TEXT']
            msg['disc'] = 0
            msg['total'] = 0

        if 'disc' in msg and 470 < row['X1'] < 500:
            msg['disc'] = row['TEXT']
            msg['total'] = 0

        if 'total' in msg and 520 < row['X1'] < 560:
            msg['total'] = row['TEXT']
            result.append(msg)
            msg = dict()

    return result


In [123]:
result = get_table_data(df)

In [124]:
import pandas as pd
new_df = pd.DataFrame(result)
new_df

,qty,part,desc,unit,disc,total
0,50,3551,Baldi Insulators 32Mm X 35Mm X,3.68539,30.00%,128.99
1,50,3569,Insulators 32Mm X 35Mm X 10Mm,3.73504,30.00%,130.72
2,25,4054,Baldi Insulators 41Mm X 40Mm X,4.52919,30.00%,79.26
3,50,4062,Baldi Insulators 41Mm X 40Mm X,4.57882,30.00%,160.26
4,50,5069,Baldi Insulators 36Mm X 50Mm X,4.72772,30.00%,165.47
5,24,7560,Baldi Insulators 50Mm X 75Mm X,8.45035,30.00%,141.97
6,8,T1EN40X100G,Panel Trunking 40x100 G N/Slot 2mt (B02582),18.17796,25.00%,109.06
7,18,T1EN40X60G,Panel Trunking 40x60 G N/Slot 2mt (B02580),8.97722,25.00%,121.19
8,8,T1EN25X100G,Panel Trunking 25x100 G N/Slot 2mt (B00846),14.05934,25.00%,84.35
9,36,T1EN25X40G,Panel Trunking 25x40 G N/Slot 2mt (B02576),5.72019,25.00%,154.45
